In [7]:
import pandas as pd
import pickle

df_files = pd.read_excel('../data/patentData_80000_20180622.xlsx', encoding='utf-8')

In [8]:
import jieba
stopwordfile = open('../dict/StopWords_CON.txt', 'r', encoding='utf-8')
def getstopword():
    w = set()
    for line in stopwordfile:
        line.strip().split('\n')
        w.add(line[:len(line)-1].strip())
    return w

stopwordset = getstopword()

def cutWords(sentence):
    word_list = jieba.cut(sentence)
    res = ' '.join(word_list)
    res = res.split(' ')
    tempX = ''
    for i in res:
        if i not in stopwordset:
            tempX+=i
            tempX+=' '
    return tempX.strip()

In [9]:
df_files['all_sentence'] = df_files['标题'] + df_files['摘要'] + df_files['首项权利要求']

In [10]:
import numpy as np
def DropNan(sentence):
    if sentence is np.nan:
        return ''
    return sentence

df_files['all_sentence'] = df_files['all_sentence'].apply(DropNan)
df_files['all_sentence'] = df_files['all_sentence'].apply(lambda x: x.replace('\n', ' '))
df_files['all_sentence'] = df_files['all_sentence'].apply(cutWords)


In [11]:
df_files['all_sentence']

0        电动 真空 镊子 电动 真空 镊子 壳体 壳体 电源 电源 接触 片 电机 串联 电机 旋转...
1        核工业 钎焊 料 系 核工业 钎焊 料 特别 核工 业 反应堆 不锈钢 耐蚀 合金 高温 合...
2        伴有 沉淀 分离 溶剂 莘取 级 效率高 适应性 强 伴有 沉淀 生 分离 沉淀 溶剂萃取 ...
3        谷物 粗粒粉 精选 清粉机 一项 谷物 粗粒粉 精选 干 叠放 筛层 筛层 至少 一台 平衡...
4        旁滤式 离心机 难 分离 细粒 粘性 固渣 高效 分离 旁滤式 离心机 该机 装有 滤饼 旁...
                               ...                        
79995    重组 干扰素 ω 非典型 肺炎 防治 药物 中 重组 干扰素 ω 防治 非典 型 肺炎 药物...
79996    ERBB 基础 肿瘤 治疗 物 治疗 哺乳动物 特别 人体 肿瘤 物 ErbB 蛋 编码 E...
79997    治疗 儿童 股 骨干 骨折 可调式 悬吊 牵引 架 治疗 儿童 股 骨干 骨折 可调式 悬吊...
79998    混合 纱布 袖管 包扎 包扎 混合 纱布 袖管 包扎 混合 纱布 袖管 柔韧 伸缩性 纤维 ...
79999    人工 食管 人工 食管 猪 主动脉 酶 解可 吸收 无损伤 羊肠线 缝制成 二个 管道 二个...
Name: all_sentence, Length: 80000, dtype: object

In [12]:
all_words = df_files['all_sentence'].values

In [13]:
word2id = {
    'padding': 0,
    'unknown': 1
}
id2word = {
    0: 'padding',
    1: 'unknown'
}
cnt = 2
for line_words in all_words:
    word_list = line_words.split(' ')
    # print (word_list)
    for word in word_list:
        # print (word)
        # print (word not in word2id.keys())
        if word not in word2id.keys():
            word2id[word] = cnt
            id2word[cnt] = word
            cnt += 1
            # print (cnt)



In [14]:
pickle.dump([word2id, id2word], open("../data/word2idandid2word.pkl","wb"))

In [15]:
from gensim.models.word2vec import Word2Vec

modelword2vec = Word2Vec.load('../model/patentTIandAB_300features_40minwords_10context.model')
word2vec_matrix = np.zeros((len(word2id), 300))
for key in id2word.keys():
    if (id2word[key] in modelword2vec):
        word2vec_matrix[key] = modelword2vec[id2word[key]]

/data/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/data/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [16]:
pickle.dump(word2vec_matrix, open("../data/word2vec_matrix.pkl","wb"))